<a href="https://colab.research.google.com/github/rahiakela/data-science-research-and-practice/blob/main/data-extraction-works/01-tabulapy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tabula-py

##Loading pdf into Pandas

In [140]:
import tabula

# Read pdf into DataFrame
dfs = tabula.read_pdf("redacted1.pdf", lattice=True, relative_area=True, multiple_tables=True, pages=2)

# convert PDF into CSV
#tabula.convert_into("redacted1.pdf", "output.csv", output_format="csv")

Oct 18, 2022 11:07:07 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



In [185]:
dfs[6]

,Unnamed: 0,Unnamed: 1,Unnamed: 2
0,NaN,NaN,NaN


In [178]:
dfs[1].columns[0]

'Encounter'

In [116]:
len(dfs[6].value_counts())

4

In [186]:
data_df = []
for idx in range(len(dfs)):
  if dfs[idx].columns[0] != "Unnamed: 0":
    data_df.append(dfs[idx])

In [187]:
len(data_df)

2

In [167]:
data_df[1]

,Test Name,Results,Reference Range,Score,Unnamed: 0,Unnamed: 1
0,HEMOGLOBIN,11.3 L,11.7-15.5 g/dL,5,NaN,NaN
1,RDW,16.5 H,11.0-15.0 %,5,NaN,NaN
2,Encounter,Specimen,Service Date,High Score,NaN,NaN
3,25,NaN,2021-06-23,5,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
5,Diagnosis Code,Description,Code Type,Score,NaN,NaN
6,D649,"Anemia, unspecified",ICD10,5,NaN,NaN
7,D709,"Neutropenia, unspecified",ICD10,5,NaN,NaN
8,Z79899,Other long term (current) drug therapy,ICD10,5,NaN,NaN
9,Test Name,Results,Reference Range,Score,NaN,NaN


##Save Dataframe into JSON file

In [168]:
# drop column that has named "Unnamed: xxx"
data_df = data_df[1][data_df[1].columns.drop(list(data_df[1].filter(regex='Unnamed')))]
data_df

,Test Name,Results,Reference Range,Score
0,HEMOGLOBIN,11.3 L,11.7-15.5 g/dL,5
1,RDW,16.5 H,11.0-15.0 %,5
2,Encounter,Specimen,Service Date,High Score
3,25,NaN,2021-06-23,5
4,NaN,NaN,NaN,NaN
5,Diagnosis Code,Description,Code Type,Score
6,D649,"Anemia, unspecified",ICD10,5
7,D709,"Neutropenia, unspecified",ICD10,5
8,Z79899,Other long term (current) drug therapy,ICD10,5
9,Test Name,Results,Reference Range,Score


In [169]:
# Drop row that has all NaN values
data_df= data_df.dropna(how="all", axis=0)
data_df

,Test Name,Results,Reference Range,Score
0,HEMOGLOBIN,11.3 L,11.7-15.5 g/dL,5
1,RDW,16.5 H,11.0-15.0 %,5
2,Encounter,Specimen,Service Date,High Score
3,25,NaN,2021-06-23,5
5,Diagnosis Code,Description,Code Type,Score
6,D649,"Anemia, unspecified",ICD10,5
7,D709,"Neutropenia, unspecified",ICD10,5
8,Z79899,Other long term (current) drug therapy,ICD10,5
9,Test Name,Results,Reference Range,Score
10,"IRON, TOTAL",31 L,40-190 mcg/dL,5


In [170]:
# save to json file
out_file = data_df.to_json('test.json', date_format='iso', orient='records')

In [171]:
!cat test.json

[{"Test Name":"HEMOGLOBIN","Results":"11.3 L","Reference Range":"11.7-15.5 g\/dL","Score":"5"},{"Test Name":"RDW","Results":"16.5 H","Reference Range":"11.0-15.0 %","Score":"5"},{"Test Name":"Encounter","Results":"Specimen","Reference Range":"Service Date","Score":"High Score"},{"Test Name":"25","Results":null,"Reference Range":"2021-06-23","Score":"5"},{"Test Name":"Diagnosis Code","Results":"Description","Reference Range":"Code Type","Score":"Score"},{"Test Name":"D649","Results":"Anemia, unspecified","Reference Range":"ICD10","Score":"5"},{"Test Name":"D709","Results":"Neutropenia, unspecified","Reference Range":"ICD10","Score":"5"},{"Test Name":"Z79899","Results":"Other long term (current) drug therapy","Reference Range":"ICD10","Score":"5"},{"Test Name":"Test Name","Results":"Results","Reference Range":"Reference Range","Score":"Score"},{"Test Name":"IRON, TOTAL","Results":"31 L","Reference Range":"40-190 mcg\/dL","Score":"5"},{"Test Name":"% SATURATION","Results":"8 L","Reference

In [172]:
def save_data_into_json(df, idx):
  # drop column that has named "Unnamed: xxx"
  data_df = df[df.columns.drop(list(df.filter(regex="Unnamed")))]
  # Drop row that has all NaN values
  data_df= data_df.dropna(how="all", axis=0)
  # save to json file
  data_df.to_json(f"data/test-{idx}.json", date_format="iso", orient="records")

In [189]:
!mkdir data

In [190]:
for idx in range(len(dfs)):
  if dfs[idx].columns[0] != "Unnamed: 0":
    save_data_into_json(dfs[idx], idx)

##Further practice

In [ ]:
tabula.read_pdf("redacted1.pdf", pages="all", stream=True)

In [72]:
dfs = tabula.read_pdf(
    "redacted1.pdf",
    lattice=True,
    #pandas_options={"header": [0, 1]},
    # area=[0, 0, 50, 100],
    relative_area=True,
    multiple_tables=True,
    pages=1,
)

Oct 18, 2022 10:37:59 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



In [75]:
df = dfs[0].drop(["Unnamed: 0"], axis=1)
df

""


In [79]:
len(df)

0

In [61]:
dfs[9]

,Unnamed: 0,Unnamed: 1
0,NaN,NaN


In [ ]:
len(dfs[6].value_counts())

4

In [62]:
data_idx = []
for idx in range(len(dfs)):
  if len(dfs[idx].value_counts()) > 0:
    data_idx.append(idx)

In [63]:
data_idx

[6, 7]

In [68]:
dfs[8]

,Encounter,Specimen,Service Date,High Score,Unnamed: 0,Unnamed: 1
0,30,NaN,2021-09-24,5,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,Diagnosis Code,Description,Code Type,Score,NaN,NaN
3,D649,"Anemia, unspecified",ICD10,5,NaN,NaN
4,E611,Iron deficiency,ICD10,5,NaN,NaN
5,I951,Orthostatic hypotension,ICD10,5,NaN,NaN
6,N920,Excessive and frequent menstruation with regul...,ICD10,5,NaN,NaN
7,R197,"Diarrhea, unspecified",ICD10,5,NaN,NaN
8,R7889,"Finding of other specified substances, not nor...",ICD10,5,NaN,NaN
9,Test Name,Results,Reference Range,Score,NaN,NaN


In [8]:
data_df = None
for idx in range(len(dfs)):
  if len(dfs[idx].value_counts()) > 0:
    data_df = dfs[idx]
data_df

,Condition,Provider\rGenerated,Demographic\rPrevalence,Confidence,Contributing Factors
0,Oth symptoms and signs w\rcognitive functions ...,False,1.4%,99.0%,"Amorphous Sediment [Presence],Platelet\r[Prese..."
1,Any Muscle Disease,True,43.5%,89.5%,"R29-Nervous Systom\rSymp.,Sodium,Hydromorphone..."
2,Abnormalities of breathing,False,5.7%,57.2%,"LE,HDL,Free T4 Index,GFR - MDRD"
3,Other anemias,True,4.0%,52.2%,"Total Bilirubin,ALP,Hemoglobin,D50-Anemia(Iron)"


In [13]:
tabula.convert_into("redacted1.pdf", "test.json", output_format="json")

Oct 18, 2022 9:53:01 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



In [14]:
!cat test.json

[{"extraction_method":"stream","top":249.0,"left":41.0,"width":523.0,"height":116.0,"right":564.0,"bottom":365.0,"data":[[{"top":256.81,"left":71.71,"width":48.05000305175781,"height":4.630000114440918,"text":"Condition"},{"top":256.81,"left":161.93,"width":43.350006103515625,"height":4.630000114440918,"text":"Provider"},{"top":256.81,"left":222.26,"width":66.68000793457031,"height":4.630000114440918,"text":"Demographic"},{"top":256.81,"left":300.38,"width":55.839996337890625,"height":4.630000114440918,"text":"Confidence"},{"top":256.81,"left":414.65,"width":98.8900146484375,"height":4.630000114440918,"text":"Contributing Factors"}],[{"top":0.0,"left":0.0,"width":0.0,"height":0.0,"text":""},{"top":267.97,"left":158.9,"width":50.850006103515625,"height":4.630000114440918,"text":"Generated"},{"top":267.97,"left":229.18,"width":52.810001373291016,"height":4.630000114440918,"text":"Prevalence"},{"top":0.0,"left":0.0,"width":0.0,"height":0.0,"text":""},{"top":0.0,"left":0.0,"width":0.0,"hei

In [24]:
tabula.convert_into("redacted1.pdf", "test.tsv", output_format="tsv", 
                    pages='all', lattice=True, area=[0, 0, 50, 100],
                    relative_area=True,
                    )

Oct 18, 2022 9:58:42 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



In [25]:
!cat test.tsv

""
""
""
""
""
""
Prevalence"	Confidence	Contributing Factors
[Presence],Methadone,ESR"
Symp.,Sodium,Hydromorphone,Free T4"
Abnormalities of breathing	False	5.7%	57.2%	LE,HDL,Free T4 Index,GFR - MDRD
Other anemias	True	4.0%	52.2%	Total Bilirubin,ALP,Hemoglobin,D50-Anemia(Iron)
""
Encounter	Specimen	Service Date	High Score		
26		2021-08-17	5		
""					
""	
""	
Test Name	Results	Reference Range	Score		
HEMOGLOBIN	11.3 L	11.7-15.5 g/dL	5		
RDW	16.5 H	11.0-15.0 %	5		
Encounter	Specimen	Service Date	High Score		
25		2021-06-23	5		
""					
Diagnosis Code	Description	Code Type	Score		
D649	Anemia, unspecified	ICD10	5		
D709	Neutropenia, unspecified	ICD10	5		
Z79899	Other long term (current) drug therapy	ICD10	5		
Test Name	Results	Reference Range	Score		
IRON, TOTAL	31 L	40-190 mcg/dL	5		
% SATURATION	8 L	16-45 % (calc)	5		
""	
""	
""
Test Name	Results	Reference Range	Score		
LEUKOCYTE ESTERASE	TRACE A	NEGATIVE	5		
Encounter	Specimen	Service Date	High Score		
22		2021-03-31	10		
""					
Diagno

In [26]:
import pandas as pd
import json

In [30]:
master_df = pd.read_csv('test.tsv', sep='\s+', encoding='utf-8', on_bad_lines='skip')

In [31]:
out_file = master_df.to_json('test.json', date_format='iso', orient='records')

In [32]:
!cat test.json

[{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":"21"},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null}